In [ ]:
import os
import cv2
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
HEIGHT = 480
WIDTH = 480
CHANNELS = 3
CLASSES = 6
top_dropout_rate = 0.2

In [ ]:
weights_path = "../input/efnet-v2-geoaug-model/EfficientNet_V2_GeoAug.h5"
hub_url = '../input/efficientnetv2-tfhub-weight-files/tfhub_models/efficientnetv2-l-21k-ft1k/feature_vector'

In [ ]:
def get_model():
    model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape = [HEIGHT, WIDTH, CHANNELS]),
            hub.KerasLayer(hub_url, trainable = True),
            Dropout(top_dropout_rate, name = "top_dropout"),
            Dense(CLASSES, activation = 'sigmoid')
        ])
    model.build((HEIGHT, WIDTH, CHANNELS))
    model.compile(optimizer ='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.summary()
    return model

In [ ]:
model = get_model()
model.load_weights(weights_path)

In [ ]:
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission

In [ ]:
def load_image(image_id):
    file_path = str(image_id)
    img = cv2.imread(test_img+'/'+file_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
def data_full_augment(image):
    
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    flag = False


    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
        flag = True
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
        flag = True
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
        flag = True


    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
        flag = True
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
        flag = True
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
        flag = True
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90
        flag = True
        
    # Crops
    if p_crop > .7:
        if p_crop > .9:
            image = tf.image.central_crop(image, central_fraction=.7)
        elif p_crop > .8:
            image = tf.image.central_crop(image, central_fraction=.8)
        else:
            image = tf.image.central_crop(image, central_fraction=.9)
        flag = True
    elif p_crop > .4:
        HEIGHT1 = image.shape[0]
        WIDTH1 = image.shape[1]
        crop_size_h = tf.random.uniform([], int(HEIGHT1*.8), HEIGHT1, dtype=tf.float32)
        crop_size_w = tf.random.uniform([], int(WIDTH1*.8), WIDTH1, dtype=tf.float32)
        image = tf.image.random_crop(image, size=[crop_size_h, crop_size_w, 3])
        flag = True
    
    return image

In [ ]:
def process(img):
    img = data_full_augment(img)
    img = tf.cast(img, tf.float32) / 255.0
    img = np.array(img)
    return cv2.resize(img , (480, 480)).reshape(-1, 480, 480, 3)
def predict(img):
    img = load_image(img)
    tta_steps = 5
    predictions = []
    for i in range(tta_steps):
        pred = model.layers[2](model.layers[1](model.layers[0](process(img)))).numpy()[0]
        predictions.append(pred)
    result = np.median(predictions, axis=0)
    
    return result

In [ ]:
test_images = submission['image'][2]
test_images = load_image(test_images)
preds = model.layers[2](model.layers[1](model.layers[0](process(test_images)))).numpy()[0]
print(preds)

In [ ]:
n_label = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
label = []
for i in range(len(submission['image'])):
    test_images = submission['image'][i]
    preds = predict(test_images)
    answer = []
    for j in range(len(preds)):
        if preds[j] > 0.4:
            answer.append(n_label[j])
    answer = ' '.join(answer)
    label.append(answer)
submission['labels'] = label
submission

In [ ]:
submission.to_csv('submission.csv', index=False)